In [18]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
from datetime import datetime,timedelta
from sklearn import preprocessing

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 6, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False  

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [26]:
knn_train = pd.read_csv('./submit/knn_train.csv')
lgb_train = pd.read_csv('./submit/lgb_train.csv')
rf_train = pd.read_csv('./submit/rf_train.csv')
ada_train = pd.read_csv('./submit/ada_train.csv')

knn_test = pd.read_csv('./submit/knn_2.csv')
lgb_test = pd.read_csv('./submit/lgb_3.csv')
rf_test = pd.read_csv('./submit/rf_2.csv')
ada_test = pd.read_csv('./submit/ada_2.csv')

train = pd.read_csv('./input/train.csv')

In [27]:
knn_train.columns = ['knn']
lgb_train.columns = ['lgb']
rf_train.columns = ['rf']
ada_train.columns = ['ada']

knn_test.columns = ['id', 'knn']
lgb_test.columns = ['id', 'lgb']
rf_test.columns = ['id', 'rf']
ada_test.columns = ['id', 'ada']

In [28]:
x_train = pd.concat([knn_train, lgb_train, rf_train, ada_train], axis=1)
del train
del knn_train
del lgb_train
del rf_train
del ada_train

x_train['target'] = target
x_train.shape
x_train.head()

x_test = pd.concat([knn_test, lgb_test, rf_test, ada_test], axis=1)
x_test = x_test.iloc[:, [0, 1, 3, 5, 7]]
del knn_test
del lgb_test
del rf_test
del ada_test

x_test.shape
x_test.head()

(7377418, 5)

,knn,lgb,rf,ada,target
0,0.533333,0.440307,0.405487,0.497449,1
1,0.466667,0.608945,0.620883,0.502656,1
2,0.600000,0.653683,0.621534,0.502864,1
3,0.400000,0.577164,0.616898,0.502628,1
4,0.600000,0.439646,0.411104,0.497871,1


(2556790, 5)

,id,knn,lgb,rf,ada
0,0,0.333333,0.619831,0.635496,0.503618
1,1,0.600000,0.687555,0.638201,0.503425
2,2,0.400000,0.194524,0.373883,0.494474
3,3,0.466667,0.206033,0.231678,0.492669
4,4,0.466667,0.199449,0.230307,0.492219


In [29]:
import xgboost as xgb

In [30]:
y_train = x_train.target
x_train = x_train.drop(['target'], axis=1)
ids = x_test.id
x_test = x_test.drop(['id'], axis=1)

In [31]:
gbm = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=500,
    max_depth=6,
    min_child_weight=1,
    gamma=0.9,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    nthread=-1,
    scale_pos_weight=1,
    seed=0)

In [32]:
gbm.fit(x_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.9, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)

In [33]:
y_test = gbm.predict_proba(x_test)

In [34]:
sub = pd.DataFrame()
sub['id'] = ids
sub['target'] = y_test[:, 1]
sub.head()

,id,target
0,0,0.406372
1,1,0.742334
2,2,0.146099
3,3,0.202254
4,4,0.185990


In [35]:
sub.to_csv('./submit/stack_xgb_2.csv', index=False, float_format = '%.6f')